<a href="https://colab.research.google.com/github/Dmitri9149/TensorFlow-PyTorch-basics/blob/master/TensorFlow_Blocks_and_Layesr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import tensorflow as tf

The code is based on the d2l.ai book http://d2l.ai/

In [45]:
### the code is based on d2l.ai book 

In [46]:
### sequence of models , Dense with relu is the first in the chain
net = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,activation=tf.nn.relu),
  tf.keras.layers.Dense(10)
])

X=tf.random.uniform((2,20))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 4.6504039e-01, -3.7268549e-04,  1.4102507e-01,  1.6386609e-01,
        -2.0623261e-01,  1.2028127e-01, -1.0512621e-01, -3.8767368e-01,
        -8.6905032e-02,  2.8686002e-02],
       [ 3.5178965e-01,  3.9137855e-02,  1.3240145e-01,  1.4432985e-01,
        -1.9109491e-01, -5.9812516e-02, -1.4314066e-01, -6.3585567e-01,
        -2.0194933e-01, -6.1389461e-02]], dtype=float32)>

In [47]:
net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 4.6504039e-01, -3.7268549e-04,  1.4102507e-01,  1.6386609e-01,
        -2.0623261e-01,  1.2028127e-01, -1.0512621e-01, -3.8767368e-01,
        -8.6905032e-02,  2.8686002e-02],
       [ 3.5178965e-01,  3.9137855e-02,  1.3240145e-01,  1.4432985e-01,
        -1.9109491e-01, -5.9812516e-02, -1.4314066e-01, -6.3585567e-01,
        -2.0194933e-01, -6.1389461e-02]], dtype=float32)>

In [48]:
### Custom Block

In [49]:
class MLP(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.hidden=tf.keras.layers.Dense(units=256,activation=tf.nn.relu)
    self.out = tf.keras.layers.Dense(units=10)

  def call(self, X):
    return self.out(self.hidden((X)))

In [50]:
net=MLP()
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.25184473, -0.10998718,  0.14494838, -0.05849118, -0.1139569 ,
         0.08733304,  0.0009399 , -0.03739362, -0.09802802,  0.14406863],
       [-0.10416563, -0.05021951,  0.14003393, -0.12758216, -0.15856056,
         0.24084733, -0.14473459,  0.04739891,  0.02985316, -0.10704499]],
      dtype=float32)>

In [51]:
class MySequential(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    for module in self.modules:
      X=module(X)
    return(X)


In [52]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[-0.00599586, -0.34658113, -0.1930099 , -0.06366614, -0.01871253,
         0.08566913,  0.03153611, -0.5249379 , -0.04460344, -0.08116271],
       [ 0.12927675, -0.36692274, -0.09881213, -0.15229964, -0.0207973 ,
        -0.28241995, -0.0886751 , -0.3907442 ,  0.05780356, -0.29689184]],
      dtype=float32)>

Below the Module is constructed where several models are applied in parallel on the same input data and the results are concatenated. It is intended to use the code in multihead attention in Transformers.

In [53]:
#initialize the list of n models which is constructed from the same model
#
def list_of_models(model, n):
  return [model]*n


In [54]:
list_models = [tf.keras.layers.Dense(10)]*3
arg_list = list_of_models(tf.keras.layers.Dense(10),3)

In [55]:
### use *args to supply a list of models of variable length 
class MyParallel(tf.keras.Model):
  def __init__(self, *args):
    super().__init__()
    self.modules = []
    for block in args:
      self.modules.append(block)

  def call(self,X):
    list_res= []
    for module in self.modules:
      list_res.append(module(X))
#      print(len(list_res))
#    print(list_res)
    concat_final= tf.concat(list_res, -1)
    return(concat_final)

In [56]:
net = MyParallel(tf.keras.layers.Dense(10), tf.keras.layers.Dense(10), tf.keras.layers.Dense(10))
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.44144467,  0.7776139 ,  0.06102974,  0.23586884,  0.84543943,
         0.5533298 , -0.23535815, -0.5041257 , -0.5208286 ,  0.38090858,
        -0.00727139,  0.7871824 , -0.9414346 , -0.7505659 , -0.58737296,
        -0.2943083 , -0.7011899 , -0.16126645,  0.00210738, -0.01336318,
         0.64622176, -0.26863432,  0.06435853,  0.20532823, -0.05624871,
         0.01881026,  0.5544808 , -0.21350528,  0.3308097 ,  0.06227104],
       [-0.09562626,  1.2704567 , -0.42853633,  0.5992415 ,  0.9173841 ,
         0.03431927, -0.00155487,  0.11077866, -0.5407151 , -0.03168237,
        -0.3400291 ,  0.9909556 , -0.32529828, -0.08113459, -0.64991343,
         0.01439545, -0.24554326, -0.10330348, -0.48539433, -0.22174594,
        -0.15891206, -0.32211384, -0.01981609,  0.66583806, -0.78658736,
         0.85270274,  0.34786147, -0.36356002,  1.1109309 , -0.24503922]],
      dtype=float32)>

In [57]:
### to use the list_models (which is a list) we have to unpack it
net = MyParallel(*list_models)
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-1.4360933 ,  0.40400538, -0.6643708 , -0.3959778 , -1.027022  ,
         0.17897028,  1.1965098 , -0.18753713,  0.38946775,  1.169302  ,
        -1.4360933 ,  0.40400538, -0.6643708 , -0.3959778 , -1.027022  ,
         0.17897028,  1.1965098 , -0.18753713,  0.38946775,  1.169302  ,
        -1.4360933 ,  0.40400538, -0.6643708 , -0.3959778 , -1.027022  ,
         0.17897028,  1.1965098 , -0.18753713,  0.38946775,  1.169302  ],
       [-1.4119146 ,  0.4496594 , -0.8920491 , -0.7407    , -0.8593414 ,
        -0.20986453,  1.8871973 , -0.43030584,  0.31833243,  1.5371679 ,
        -1.4119146 ,  0.4496594 , -0.8920491 , -0.7407    , -0.8593414 ,
        -0.20986453,  1.8871973 , -0.43030584,  0.31833243,  1.5371679 ,
        -1.4119146 ,  0.4496594 , -0.8920491 , -0.7407    , -0.8593414 ,
        -0.20986453,  1.8871973 , -0.43030584,  0.31833243,  1.5371679 ]],
      dtype=float32)>

In [58]:
net = MyParallel(*arg_list)
X = tf.random.uniform((2,10))
net(X)

<tf.Tensor: shape=(2, 30), dtype=float32, numpy=
array([[-0.685372  , -0.20500517, -0.3766685 ,  0.76773334, -0.07041395,
        -0.16667587, -0.36151233,  0.80915624, -0.24390078,  0.01573527,
        -0.685372  , -0.20500517, -0.3766685 ,  0.76773334, -0.07041395,
        -0.16667587, -0.36151233,  0.80915624, -0.24390078,  0.01573527,
        -0.685372  , -0.20500517, -0.3766685 ,  0.76773334, -0.07041395,
        -0.16667587, -0.36151233,  0.80915624, -0.24390078,  0.01573527],
       [-1.2358997 , -0.21316257,  0.07162188,  0.843853  , -0.1167118 ,
        -0.31461346, -0.93894285,  1.0720873 , -0.1310479 , -0.32373545,
        -1.2358997 , -0.21316257,  0.07162188,  0.843853  , -0.1167118 ,
        -0.31461346, -0.93894285,  1.0720873 , -0.1310479 , -0.32373545,
        -1.2358997 , -0.21316257,  0.07162188,  0.843853  , -0.1167118 ,
        -0.31461346, -0.93894285,  1.0720873 , -0.1310479 , -0.32373545]],
      dtype=float32)>

In [59]:
### Nested Models 
class NestedBlocks(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.net=tf.keras.Sequential()
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.net.add(tf.keras.layers.Dense(20,activation = tf.nn.relu))
    self.dense = tf.keras.layers.Dense(20,activation = tf.nn.relu)

  def call(self, inputs):
    return self.dense(self.net(inputs))



In [60]:
net = NestedBlocks()
X = tf.random.uniform((2, 20))
net(X)

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.        , 0.107412  , 0.278081  , 0.        , 0.07245494,
        0.        , 0.        , 0.        , 0.        , 0.20427996,
        0.        , 0.01383293, 0.        , 0.2972421 , 0.11352599,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.02782123, 0.05398005, 0.        , 0.        ,
        0.05253005, 0.        , 0.10640791, 0.        , 0.11002039,
        0.        , 0.00583928, 0.        , 0.33761588, 0.17971468,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>

Parameter access

In [61]:
import numpy as np

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1)
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.31627157],
       [-0.01517489]], dtype=float32)>

In [62]:
print(net.layers[2].weights)

[<tf.Variable 'dense_51/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[-0.2893197 ],
       [ 0.54835355],
       [ 0.1897304 ],
       [-0.03031921]], dtype=float32)>, <tf.Variable 'dense_51/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


In [63]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'tensorflow.python.ops.resource_variable_ops.ResourceVariable'>
<tf.Variable 'dense_51/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>
tf.Tensor([0.], shape=(1,), dtype=float32)


In [64]:
print(net.layers[1].weights)
print(net.get_weights())

[<tf.Variable 'dense_50/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.01635313, -0.21623337, -0.59352034,  0.8598476 ],
       [-0.24274784,  0.8045687 ,  0.70814615,  0.19596893],
       [ 0.82256955,  0.19033974,  0.19821459, -0.04185528],
       [-0.3091538 , -0.4348423 ,  0.5587905 , -0.26054966]],
      dtype=float32)>, <tf.Variable 'dense_50/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]
[array([[-0.01635313, -0.21623337, -0.59352034,  0.8598476 ],
       [-0.24274784,  0.8045687 ,  0.70814615,  0.19596893],
       [ 0.82256955,  0.19033974,  0.19821459, -0.04185528],
       [-0.3091538 , -0.4348423 ,  0.5587905 , -0.26054966]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.2893197 ],
       [ 0.54835355],
       [ 0.1897304 ],
       [-0.03031921]], dtype=float32), array([0.], dtype=float32)]


In [65]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

In [66]:
### collecting parameters from nested blocks
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # Nested here
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.00305007],
       [0.00149557]], dtype=float32)>

In [67]:
print(rgnet.summary())

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_15 (Sequential)   (2, 4)                    80        
_________________________________________________________________
dense_56 (Dense)             (2, 1)                    5         
Total params: 85
Trainable params: 85
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<tf.Variable 'sequential_15/block-1/dense_53/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>

In [69]:
net_prl = MyParallel(tf.keras.layers.Dense(4), tf.keras.layers.Dense(4), tf.keras.layers.Dense(4))
X = tf.random.uniform((1,4))
net_prl(X)

<tf.Tensor: shape=(1, 12), dtype=float32, numpy=
array([[ 0.48810187, -0.4404224 ,  0.5570709 , -0.5339688 ,  0.94602567,
         0.7709578 , -0.6777719 ,  0.45327234, -0.9039616 , -0.4505682 ,
        -0.5535219 ,  1.1515251 ]], dtype=float32)>

In [70]:
print(net_prl.summary())

Model: "my_parallel_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_57 (Dense)             multiple                  20        
_________________________________________________________________
dense_58 (Dense)             multiple                  20        
_________________________________________________________________
dense_59 (Dense)             multiple                  20        
Total params: 60
Trainable params: 60
Non-trainable params: 0
_________________________________________________________________
None


In [71]:
net = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(
      4, activation = tf.nn.relu, 
      kernel_initializer = tf.random_normal_initializer(mean=0, stddev=0.01),
      bias_initializer=tf.zeros_initializer()
  ),
  tf.keras.layers.Dense(1)                                  
])
net(X)
net.weights

[<tf.Variable 'dense_60/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.00401406, -0.01121294, -0.00982633, -0.01760029],
        [-0.01375725,  0.00803078, -0.01387203, -0.00121617],
        [ 0.01031441, -0.00690931,  0.01110723,  0.00201837],
        [-0.00087148, -0.01173297, -0.01783088, -0.00239353]],
       dtype=float32)>,
 <tf.Variable 'dense_60/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_61/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[-0.66132045],
        [ 0.05364573],
        [-0.8701498 ],
        [ 0.23759663]], dtype=float32)>,
 <tf.Variable 'dense_61/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [72]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<tf.Variable 'dense_62/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=float32)>,
 <tf.Variable 'dense_62/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

In [73]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(1)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<tf.Variable 'dense_64/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[-0.4967421 , -0.4775313 , -0.703524  ,  0.37484747],
       [-0.6972176 ,  0.32043463, -0.5647408 , -0.8072155 ],
       [-0.34709018, -0.5511613 , -0.19392627, -0.2540849 ],
       [-0.09782797,  0.44074732,  0.5893937 ,  0.6423599 ]],
      dtype=float32)>
<tf.Variable 'dense_65/kernel:0' shape=(4, 1) dtype=float32, numpy=
array([[1.],
       [1.],
       [1.],
       [1.]], dtype=float32)>


In [74]:
class MyInit(tf.keras.initializers.Initializer):
  def __init__(self,shape, dtype=None):
    data = tf.random.uniform(shape, -10,10,dtype=dtype)
    factor=(tf.abs.data >=5)
    factor=tf.cast(factor, tf.float32)
    return factor*data

net(X)
net.layers[1].weights

[<tf.Variable 'dense_64/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.4967421 , -0.4775313 , -0.703524  ,  0.37484747],
        [-0.6972176 ,  0.32043463, -0.5647408 , -0.8072155 ],
        [-0.34709018, -0.5511613 , -0.19392627, -0.2540849 ],
        [-0.09782797,  0.44074732,  0.5893937 ,  0.6423599 ]],
       dtype=float32)>,
 <tf.Variable 'dense_64/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

In [75]:
### assign parameters
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<tf.Variable 'dense_64/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[42.        ,  0.5224687 ,  0.296476  ,  1.3748474 ],
       [ 0.30278242,  1.3204346 ,  0.43525922,  0.19278449],
       [ 0.6529098 ,  0.4488387 ,  0.8060737 ,  0.7459151 ],
       [ 0.902172  ,  1.4407473 ,  1.5893936 ,  1.64236   ]],
      dtype=float32)>

In [76]:
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

True
